### using simple decision tree function predicting survival on Titanic. This model got 76% rate success on kaggle 

In [1]:
import pandas as pd

In [2]:
titanic_df = pd.read_csv('train.csv')

In [3]:
titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [4]:
titanic_df.shape

(891, 12)

In [5]:
titanic_df['AgeRange'] = pd.cut(titanic_df['Age'], [0, 15, 80], labels=['child', 'adult'])

In [6]:
# 38% survival rate from train data
titanic_df['Survived'].mean()

0.3838383838383838

In [7]:
titanic_df.drop(['PassengerId', 'Name', 'SibSp', 'Parch', 'Ticket', 'Cabin'], axis=1, inplace=True)

In [8]:
titanic_df_clean_age = titanic_df.dropna(subset=['Age'])

In [9]:
# Outcome of survival rate of group of data. 
grouped_data = pd.concat(
    [titanic_df_clean_age.groupby(['Pclass', 'Sex', 'AgeRange'])['Survived'].mean(),
     titanic_df_clean_age.groupby(['Pclass', 'Sex', 'AgeRange'])['Survived'].count()],
    axis=1)
grouped_data.columns = ['Survived', 'Count']
grouped_data

Survived  Count
Pclass Sex    AgeRange                 
1      female child     0.666667      3
              adult     0.975610     82
       male   child     1.000000      3
              adult     0.377551     98
2      female child     1.000000     10
              adult     0.906250     64
       male   child     1.000000      9
              adult     0.066667     90
3      female child     0.533333     30
              adult     0.430556     72
       male   child     0.321429     28
              adult     0.128889    225

### Above We can see that PClass 1 and 2 women have more than 50% survival chance, and childrens below age 15 too. Using this data we will make a simple decision tree model.

In [10]:
# survived or not data in series
real = titanic_df_clean_age['Survived']

In [12]:
real.mean()

0.4061624649859944

In [13]:
# Function to calculate the accuracy of our prediction. 
def accuracy(predicted,real = real):
    '''
    Take 2 series data frame 1. predicted survival from our model
    2. real survived or not and return the accuracy of our prediction.
    '''
    if len(real)==len(predicted):
        return (real==predicted).mean()
    else:
        return 'not of same length'

In [14]:
# simple survie or not prediction algorithim using decision tree.
# this algorithim improved by testing different prediction accuracies.
def prediction(titanic):
    '''
    Take data set and return a series data set
    after predicting.
    '''
    predicted=[]
    for _,passenger in titanic.iterrows():
        if passenger['Pclass'] !=3:
            if passenger['Sex']=='female':
                predicted.append(1)
            elif passenger['Age'] <= 15:
                predicted.append(1)
            else:
                predicted.append(0)
        else:
            predicted.append(0)
                
    return pd.Series(predicted)

In [15]:
train_predicted = prediction(titanic_df_clean_age)

In [16]:
print(accuracy(train_predicted))

0.8081232493


### We made a model with 80% accuracy with train data. Lets check the predictions with test data

In [17]:
test_data = pd.read_csv('test.csv')

In [18]:
predict_test = prediction(test_data)

In [19]:
test_data['Survived'] = predict_test

In [21]:
# making df for kaggle 
kaggle_df =test_data[['PassengerId','Survived']]

In [22]:
kaggle_df.to_csv('outcome.csv',index=False)